In [6]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set_style("darkgrid")
from sklearn import linear_model
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import random

import sys
import os

import warnings
warnings.filterwarnings('ignore')

#Auxiliares

def far_to_cel(temp):
    return 5*(temp)/9 - 5*32/9


#Función para sacar outliers
#(se puede cambiar eso de s-s.mean <= etc, esa es una forma posible de detectar outliers)

def mascara_outliers(s, mult):
    return abs(s - s.mean()) <= mult*s.std()

def sacar_outliers(datos,tag,mult):
    mascara = mascara_outliers(datos[tag], mult)
    return datos[mascara]


#Función para delimitar ejes:
#(dice dominio, pero se puede usar para imagen también... ya se que esta medio cabeza, pero anda bien)

def mascara_dominio_abajo(s,x0):
    return x0 <= s
def mascara_dominio_arriba(s,x1):
    return s <= x1

def achicar_dominio(datos,tag,x0,x1):
    mascara = mascara_dominio_abajo(datos[tag],x0)
    datos = datos[mascara]
    mascara = mascara_dominio_arriba(datos[tag],x1)
    return datos[mascara]


#Función para crear una columna con cada año
#(es imrpotante haber cargado el csv con el arguemtno parse_dates=[tag])
def crear_col_años(datos,tag):
    return datos[tag].apply (lambda x : x.year if x.month <= 12 else x.year + 1)


#Cuadrados mínimos para predecir

def armar_matriz_A(s):
    temp = []
    for t in s:
        #polo sur
        #temp.append([t**4, t**3, t**2, t, 1, np.sin(-70*t-1000)]) #v1
        temp.append([t^3, t^2, t, 1, np.cos(t)])                  #v2
        
        #nivel del agua zoom
        #temp.append([t, 1, np.sin(1000*t)/np.cos(t), np.sin(0.323*t)]) #v1
        #temp.append([t**3, t**2, t, 1, np.sin((0.31)*t + 0.2345)])          #v2
        

    return np.array(temp)

def entrenar_y_predecir_en_rangos(df, rango_entrenamiento, rango_prediccion, tag1, tag2):
    regr = linear_model.LinearRegression(fit_intercept=False)

    # Entreno el modelo
    df_e = df[df[tag1].isin(rango_entrenamiento)]
    A_e = armar_matriz_A(df_e[tag1])
    regr.fit(A_e, df_e[tag2])

    # Me fijo la aproximación que se realizó
    df_e['p'] = regr.predict(A_e)
    sns.tsplot(time=df_e[tag1], data=df_e['p'], color='r')

    # Realizo predicciones
    df_p = df[df[tag1].isin(rango_prediccion)]

    A_p = armar_matriz_A(df_p[tag1])
    df_p['p'] = regr.predict(A_p)
    sns.tsplot(time=df_p[tag1], data=df_p['p'], color='g')

    ECM = sum((df_p['p']-df_p[tag2])**2)
    return ECM

def predecir(datos, k, rango,limitador, tag1, tag2):
    return entrenar_y_predecir_en_rangos(datos, range(k-rango,k), range(k,k+rango-limitador), tag1, tag2)


def graficador(datos, año, rango, limitador, x0, x1, tag1, tag2, title, xl, yl):
    ECM = predecir(datos,año,rango,limitador,tag2, tag1)
    print("Error de predicción: {:.2f}".format(ECM))
    
    sns.plt.title(title)
    sns.plt.xlabel(xl)
    sns.plt.ylabel(yl)
    
    sns.plt.xlim((x0,x1))
    sns.plt.show()
    sns.plt.close()


#Auxiliares

def descripcion(datos):
    print(datos.describe())

def ver_filas(datos, fil):
    print(datos.head(fil))

In [41]:

################################
# Levanto los datos, y los veo #
################################

df_todos = []
df_nombres = []
for filename in sorted(os.listdir('./')):
    primero = True
    if not('ipynb' in filename):
        df_nombres.append(filename)
        for res in os.listdir('./'+str(filename)):
            df_actual = pd.read_csv('./'+str(filename)+'/'+str(res))
            if primero:
                primero = False
                df_todos.append(df_actual)
            else:
                df_todos[-1]['tiempo'] = df_todos[-1]['tiempo'] + df_actual['tiempo']

for i in range(0,len(df_todos)):
    df_todos[i]['tiempo'] = df_todos[i]['tiempo'] / 100

for i in range(0,len(df_todos)):
    print(df_nombres[i])
    ver_filas(df_todos[i],1)

agentesVariados_conPodaFuturo
   tam    tiempo  agentes  votos
0    1  0.000005        1      0
agentesVariados_conPodaMax
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
agentesVariados_conPodas
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
agentesVariados_sinPodas
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
cerosVariados_conPodaFuturo
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
cerosVariados_conPodas
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
cerosVariados_sinPodas
   tam    tiempo  agentes  votos
0    1  0.000004        1      0
votosVariados_conPodaFuturo
   tam    tiempo  agentes  votos
0    0  0.000077       20    400
votosVariados_conPodaMax
   tam    tiempo  agentes  votos
0    0  0.000079       20    400
votosVariados_conPodas
   tam    tiempo  agentes  votos
0    0  0.000078       20    400
votosVariados_sinPodas
   tam    tiempo  agentes  votos
0    0  0.000077       20  